## YOLOv1 구현

In [1]:
import numpy as np
import cv2
from functools import partial
import xmltodict
from tqdm import tqdm
import tensorflow as tf
import random
from glob import glob

In [2]:
max_num = len(tf.keras.applications.VGG16(weights='imagenet', include_top=False,  input_shape=(224, 224, 3)).layers) # 레이어 최대 개수

YOLO = tf.keras.models.Sequential()
for i in range(0, max_num-1):
    YOLO.add(tf.keras.applications.VGG16(weights='imagenet', include_top=False,  input_shape=(224, 224, 3)).layers[i])

initializer = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.01, seed=None)
regularizer = tf.keras.regularizers.l2(0.0005)

for layer in YOLO.layers:
    # 'kernel_regularizer' 속성이 있는 인스턴스를 찾아 regularizer를 추가
    if hasattr(layer, 'kernel_regularizer'):
        setattr(layer, 'kernel_regularizer', regularizer)

### 원문은 DarkNet을 썼지만 나는 구현을 쉽게 하기 위해 VGG16을 썼다.
### 여기에 따로 레이어를 얹어서 YOLO를 구현할거다

In [4]:
YOLO.add(tf.keras.layers.Conv2D(1024, (3, 3), kernel_initializer=initializer, padding = 'SAME' ,kernel_regularizer = regularizer, dtype='float32'))
YOLO.add(tf.keras.layers.Conv2D(1024, (3, 3), kernel_initializer=initializer, padding = 'SAME' ,kernel_regularizer = regularizer, dtype='float32'))
YOLO.add(tf.keras.layers.MaxPool2D((2, 2)))
YOLO.add(tf.keras.layers.Conv2D(1024, (3, 3), kernel_initializer=initializer, padding = 'SAME' ,kernel_regularizer = regularizer, dtype='float32'))
YOLO.add(tf.keras.layers.Conv2D(1024, (3, 3), kernel_initializer=initializer, padding = 'SAME' ,kernel_regularizer = regularizer, dtype='float32'))
# Linear 부분
YOLO.add(tf.keras.layers.Flatten())
YOLO.add(tf.keras.layers.Dense(4096, activation= None, kernel_initializer = initializer, dtype='float32'))
YOLO.add(tf.keras.layers.Dense(1470, activation=partial(tf.nn.leaky_relu, alpha=0.01), kernel_initializer = initializer, dtype='float32')) # 30*7*7 = 1470. 0~29 : (0, 0) 위치의 픽셀에 대한 각종 출력값, 30~59 : (1, 0) 위치의...블라블라

In [5]:
YOLO.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2